In [1]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'

In [2]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
split = 1

In [4]:
adata_preds = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions_{split}.h5ad")

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [5]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [6]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [7]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [8]:
adata_pred_test = adata_preds[adata_preds.obs["split"]=="test"]
adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["CPA_pred"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

adata_pred_ood = adata_preds[adata_preds.obs["split"]=="ood"]
adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["CPA_pred"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

/tmp/ipykernel_1717264/2350350615.py:2: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
/tmp/ipykernel_1717264/2350350615.py:18: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]


In [9]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [10]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [11]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [12]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [13]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [14]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.7621291960988726,
 'decoded_ood_sinkhorn_div_1': 103.31104278564453,
 'decoded_ood_sinkhorn_div_10': 70.25890186854771,
 'decoded_ood_sinkhorn_div_100': 23.84189714704241,
 'decoded_ood_e_distance': 46.90050808790995,
 'decoded_ood_mmd': 0.21227030456066132}

In [15]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7651808013518652,
 'decoded_test_e_distance': 45.630455323471466,
 'decoded_test_mmd_distance': 0.21416130103170872}

In [16]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.7621291960988726,
 'decoded_ood_sinkhorn_div_1': 103.31104278564453,
 'decoded_ood_sinkhorn_div_10': 70.25890186854771,
 'decoded_ood_sinkhorn_div_100': 23.84189714704241,
 'decoded_ood_e_distance': 46.90050808790995,
 'decoded_ood_mmd': 0.21227030456066132}

In [17]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': -0.13678244182041713,
 'encoded_ood_sinkhorn_div_1': 27.111651148114884,
 'encoded_ood_sinkhorn_div_10': 24.0260101045881,
 'encoded_ood_sinkhorn_div_100': 22.844025203159877,
 'encoded_ood_e_distance': 45.406468083834376,
 'encoded_ood_mmd': 0.20101855375937053}

In [18]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.5639298983982631,
 'deg_ood_sinkhorn_div_1': 22.153757776532853,
 'deg_ood_sinkhorn_div_10': 14.091681207929339,
 'deg_ood_sinkhorn_div_100': 13.485836914607457,
 'deg_ood_e_distance': 26.848364719616963,
 'deg_ood_mmd': 0.3010308210338865}

In [19]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.5936812311410904,
 'deg_test_sinkhorn_div_1': 22.661864459514618,
 'deg_test_sinkhorn_div_10': 14.236165245374044,
 'deg_test_sinkhorn_div_100': 13.17787996927897,
 'deg_test_e_distance': 26.203194033605445,
 'deg_test_mmd': 0.2775788524498542}

In [20]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7651808013518652,
 'decoded_test_e_distance': 45.630455323471466,
 'decoded_test_mmd_distance': 0.21416130103170872}

In [21]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.7621291960988726,
 'decoded_ood_sinkhorn_div_1': 103.31104278564453,
 'decoded_ood_sinkhorn_div_10': 70.25890186854771,
 'decoded_ood_sinkhorn_div_100': 23.84189714704241,
 'decoded_ood_e_distance': 46.90050808790995,
 'decoded_ood_mmd': 0.21227030456066132}

In [22]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.5639298983982631,
 'deg_ood_sinkhorn_div_1': 22.153757776532853,
 'deg_ood_sinkhorn_div_10': 14.091681207929339,
 'deg_ood_sinkhorn_div_100': 13.485836914607457,
 'deg_ood_e_distance': 26.848364719616963,
 'deg_ood_mmd': 0.3010308210338865}

In [23]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7651808013518652,
 'decoded_test_e_distance': 45.630455323471466,
 'decoded_test_mmd_distance': 0.21416130103170872}

In [24]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -0.4347736872231402,
 'encoded_test_sinkhorn_div_1': 26.044574836889904,
 'encoded_test_sinkhorn_div_10': 22.471602002779644,
 'encoded_test_sinkhorn_div_100': 21.305325547854107,
 'encoded_test_e_distance': 42.35948530164802,
 'encoded_test_mmd': 0.21058489506443342}

In [25]:
ood_metrics_encoded

{'Givinostat+Dasatinib': {'r_squared': 0.6846615076065063,
  'sinkhorn_div_1': 5.343008995056152,
  'sinkhorn_div_10': 3.071918487548828,
  'sinkhorn_div_100': 2.3798961639404297,
  'e_distance': 4.602572129521414,
  'mmd': 0.08602876},
 'Givinostat+SRT1720': {'r_squared': 0.6621983647346497,
  'sinkhorn_div_1': 5.196928977966309,
  'sinkhorn_div_10': 3.075993537902832,
  'sinkhorn_div_100': 2.4432497024536133,
  'e_distance': 4.748518113319809,
  'mmd': 0.08487118},
 'Panobinostat+Dasatinib': {'r_squared': -1.025991678237915,
  'sinkhorn_div_1': 50.43031311035156,
  'sinkhorn_div_10': 46.467124938964844,
  'sinkhorn_div_100': 44.8580207824707,
  'e_distance': 89.33714946295696,
  'mmd': 0.33266452},
 'Panobinostat+PCI-34051': {'r_squared': -1.1426193714141846,
  'sinkhorn_div_1': 58.21187210083008,
  'sinkhorn_div_10': 53.71614456176758,
  'sinkhorn_div_100': 51.65331268310547,
  'e_distance': 102.7907179132888,
  'mmd': 0.3477974},
 'Panobinostat+SRT2104': {'r_squared': -1.4691710472

In [26]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/cpa"

In [27]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
